In [42]:
import os
if not "AZURE_OPEN_API_KEY" in os.environ:
    os.environ["AZURE_OPEN_API_KEY"] = "PUT YOUR KEY HERE"
if not "AZURE_OPENAI_ENDPOINT" in os.environ:
    os.environ["AZURE_OPENAI_ENDPOINT"] = "PUT YOUR ENDPOINT HERE"

print("Environmental Variables Set?",  "AZURE_OPEN_API_KEY" in os.environ and "AZURE_OPENAI_ENDPOINT" in os.environ)

Environmental Variables Set? True


### Get Environmental Variables

In [40]:
API_KEY = os.environ["AZURE_OPEN_API_KEY"]
API_KEY

'9944c73f8c8945c98b0a4ebe5841006d'

In [43]:
RESOUCE_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
RESOUCE_ENDPOINT

'https://azure-openai-testx22.openai.azure.com/'

In [44]:
def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])   

### Install Packages, if Needed

In [ ]:
#pip install requests num2words pandas numpy tiktoken openai 
%pip install azure-search-documents==11.4.0b8
%pip install azure-identity
%pip install openai
%pip install scikit-learn
%pip install urllib


import_or_install('requests')
import_or_install('openai')
import_or_install('re')
import_or_install('sys')
import_or_install('num2words')
import_or_install('tiktoken')


### Helper Functions

In [69]:
# Helper Functions, for OpenAI 1.0+
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# 1.0 Version, may be helpful
#def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
#     return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    return s

#### Set a "Client" Variable To Hold All Connection Information. This is used to get embeddings from the AI Model

In [72]:
client = AzureOpenAI(
  api_key = API_KEY,  
  api_version = "2023-05-15",
  azure_endpoint = RESOUCE_ENDPOINT
)

### Load Libraries (OpenAI 1.0+)

In [45]:
import re
import requests
import sys
from num2words import num2words
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

##### Set Pandas Options

In [63]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

### Load Data

In [46]:
outfile = "bill_sum_data.csv"
if os.path.isfile(outfile):
    df = pd.read_csv(outfile)
else: 
    df = pd.read_csv("https://raw.githubusercontent.com/Azure-Samples/Azure-OpenAI-Docs-Samples/main/Samples/Tutorials/Embeddings/data/bill_sum_data.csv")
    # Write out a local copy
    df.to_csv(outfile)
    
# Import Data
#df=pd.read_csv(os.path.join(os.getcwd(),'bill_sum_data.csv')) # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
#df

#### Subset Data

In [59]:
df_bills = df[['text', 'summary', 'title']]
#df_bills.head()

#### View Data

##### Main Data

In [54]:
#df.iloc[0:5,0:len(df.columns)]
df.head()

,Unnamed: 0.1,Unnamed: 0,bill_id,text,summary,title,text_len,sum_len
0,0,0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,1,1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,2,2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,3,3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,4,4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278


##### Billing Data

In [60]:
df_bills.head()

,text,summary,title
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...


#### Confirm API is Active On Azure

In [56]:

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2023-03-15-preview"

url = openai.api_base + "/openai/deployments?api-version=2023-03-15-preview" 

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt-35-turbo",
      "status": "succeeded",
      "created_at": 1702485375,
      "updated_at": 1702485375,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002",
      "status": "succeeded",
      "created_at": 1702485403,
      "updated_at": 1702485403,
      "object": "deployment"
    }
  ],
  "object": "list"
}


#### Normalize Text For Embeddings

In [64]:
df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))

#### Remove any bills that are too long for the token limit (8192)

In [65]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

20

In [66]:
df_bills.head()

,text,summary,title,n_tokens
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038


#### To understand how many tokens we may have, look at the following:

In [67]:
sample_encode = tokenizer.encode(df_bills.text[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'SECTION',
 b' ',
 b'1',
 b'.',
 b' SHORT',
 b' TITLE',
 b'.',
 b' This',
 b' Act',
 b' may',
 b' be',
 b' cited',
 b' as',
 b' the',
 b' ``',
 b'National',
 b' Science',
 b' Education',
 b' Tax',
 b' In',
 b'cent',
 b'ive',
 b' for',
 b' Businesses',
 b' Act',
 b' of',
 b' ',
 b'200',
 b'7',
 b"''.",
 b' SEC',
 b'.',
 b' ',
 b'2',
 b'.',
 b' C',
 b'RED',
 b'ITS',
 b' FOR',
 b' CERT',
 b'AIN',
 b' CONTRIBUT',
 b'IONS',
 b' BEN',
 b'EF',
 b'IT',
 b'ING',
 b' SC',
 b'IENCE',
 b',',
 b' TECHNO',
 b'LOGY',
 b',',
 b' ENGINE',
 b'ERING',
 b',',
 b' AND',
 b' MAT',
 b'HE',
 b'MAT',
 b'ICS',
 b' EDUC',
 b'ATION',
 b' AT',
 b' THE',
 b' ELEMENT',
 b'ARY',
 b' AND',
 b' SECOND',
 b'ARY',
 b' SCHOOL',
 b' LEVEL',
 b'.',
 b' (',
 b'a',
 b')',
 b' In',
 b' General',
 b'.--',
 b'Sub',
 b'part',
 b' D',
 b' of',
 b' part',
 b' IV',
 b' of',
 b' sub',
 b'chapter',
 b' A',
 b' of',
 b' chapter',
 b' ',
 b'1',
 b' of',
 b' the',
 b' Internal',
 b' Revenue',
 b' Code',
 b' of',
 b' ',
 b'198',
 b'6',


In [68]:
len(decode)

1466

#### Generate Embeddings Via ADA-002 Deployed AI model in Azure 

In [74]:
df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings(x, model = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

#### Take A Peek

In [75]:
df_bills.head()

,text,summary,title,n_tokens,ada_v2
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,1466,"[-0.022678455337882042, -0.00799493957310915, ..."
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183,"[-0.041596777737140656, -0.009042778052389622,..."
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,937,"[-0.0420723482966423, -0.0023871997836977243, ..."
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,3670,"[-0.013265267945826054, -0.007186719682067633,..."
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,1038,"[-0.03917262703180313, -0.025464843958616257, ..."


In [76]:





res = search_docs(df_bills, "Can I get information on cable company tax revenue?", top_n=4)

,text,summary,title,n_tokens,ada_v2,similarities
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,947,"[-0.018460797145962715, -0.024794351309537888,...",0.819775
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,1183,"[-0.041596777737140656, -0.009042778052389622,...",0.734921
11,SECTION 1. SHORT TITLE. This Act may be cited ...,Wall Street Compensation Reform Act of 2010 - ...,A bill to amend the Internal Revenue Code of 1...,2331,"[-0.047416992485523224, -0.007824326865375042,...",0.734551
19,SECTION 1. SHORT TITLE. This Act may be cited ...,Strengthening the Health Care Safety Net Act o...,To amend title XIX of the Social Security Act ...,2678,"[-0.017817417159676552, -0.007099970709532499,...",0.733241


In [77]:
res["summary"][9]

"Taxpayer's Right to View Act of 1993 - Amends the Communications Act of 1934 to prohibit a cable operator from assessing separate charges for any video programming of a sporting, theatrical, or other entertainment event if that event is performed at a facility constructed, renovated, or maintained with tax revenues or by an organization that receives public financial support. Authorizes the Federal Communications Commission and local franchising authorities to make determinations concerning the applicability of such prohibition. Sets forth conditions under which a facility is considered to have been constructed, maintained, or renovated with tax revenues. Considers events performed by nonprofit or public organizations that receive tax subsidies to be subject to this Act if the event is sponsored by, or includes the participation of a team that is part of, a tax exempt organization."